# LangChain SystemMessage로 AI 페르소나 설정하기

이 노트북에서는 **SystemMessage**를 사용하여 AI의 역할과 행동 방식을 정의하는 방법을 알아봅니다.

## SystemMessage란?

SystemMessage는 AI에게 "어떤 역할을 수행할지" 알려주는 메시지입니다. 대화가 시작되기 전에 AI의 성격, 말투, 제약사항 등을 설정합니다.

**메시지 타입 비교:**

| 메시지 타입 | 역할 | 예시 |
|------------|------|------|
| `SystemMessage` | AI의 역할/성격 설정 | "당신은 친절한 고객상담원입니다" |
| `HumanMessage` | 사용자 입력 | "환불 방법을 알려주세요" |
| `AIMessage` | AI 응답 | "환불은 다음과 같이..." |

**활용 예시:**
- 특정 말투로 대답하게 하기 (존댓말, 이모지 사용 등)
- 전문 분야 역할 부여 (의사, 변호사, 교사 등)
- 응답 형식 지정 (JSON, 마크다운, 리스트 등)
- 제약사항 설정 (특정 주제만 답변, 짧게 답변 등)

---

# 1. Ollama 설치 및 서버 실행

In [1]:
import subprocess
import time

# zstd 설치 (Ollama 설치의 사전 요구 사항)
!apt-get install -y zstd

# Ollama 설치
!curl -fsSL https://ollama.com/install.sh | sh

# 백그라운드에서 Ollama 서버 실행
subprocess.Popen(['ollama', 'serve'])

time.sleep(3)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 1s (618 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...
>>> Installing ollama to /usr/local
>>> Downloading ollama-linux-amd64.tar.zst
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current use

# 2. 모델 다운로드 & 패키지 설치

- `ollama pull llama3.2` - Llama 3.2 모델 다운로드
- `pip install langchain-ollama` - LangChain Ollama 통합 패키지 설치

In [2]:
!ollama pull llama3.2
!pip install -q langchain-ollama

# 3. SystemMessage 코드 실행

**코드 구조 설명:**

### 1) 메시지 클래스 import
```python
from langchain_core.messages import HumanMessage, SystemMessage
```

### 2) SystemMessage 생성
```python
system_msg = SystemMessage('당신은 ... 어시스턴트입니다.')
```
- AI의 역할과 행동 규칙을 정의
- 이 예제에서는 "문장 끝에 느낌표 3개 붙이기" 규칙 설정

### 3) HumanMessage 생성
```python
human_msg = HumanMessage('프랑스의 수도는 어디인가요?')
```
- 사용자의 실제 질문

### 4) 메시지 리스트로 전달
```python
response = model.invoke([system_msg, human_msg])
```
- **순서 중요**: SystemMessage → HumanMessage 순으로 전달
- 모델은 System 지시사항을 따라 Human 메시지에 응답

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama

model = ChatOllama(model='llama3.2')

system_msg = SystemMessage(
    '''당신은 문장 끝에 느낌표를 세 개 붙여 대답하는 친절한 어시스턴트입니다.'''
)
human_msg = HumanMessage('프랑스의 수도는 어디인가요?')

response = model.invoke([system_msg, human_msg])
print(response.content)

파리...


---

## OpenAI에서 Ollama로 변경 시 코드 차이

```python
# OpenAI 사용 시
from langchain_openai.chat_models import ChatOpenAI
model = ChatOpenAI()

# Ollama 사용 시
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

> LangChain의 장점: 모델 클래스만 바꾸면 나머지 코드(메시지 구성, invoke 호출)는 동일하게 동작합니다.

## SystemMessage 작성 팁

1. **명확하게 작성**: "~처럼 행동하세요" 보다 구체적인 지시가 효과적
2. **제약사항 명시**: "~하지 마세요" 형태의 금지 규칙도 포함
3. **예시 포함**: 원하는 출력 형식의 예시를 보여주면 더 정확함
4. **간결하게 유지**: 너무 긴 시스템 메시지는 오히려 혼란을 줄 수 있음

## 다양한 SystemMessage 예시

```python
# 전문가 역할
SystemMessage("당신은 10년 경력의 파이썬 개발자입니다. 코드 리뷰를 해주세요.")

# 응답 형식 지정
SystemMessage("모든 답변을 마크다운 리스트 형식으로 작성하세요.")

# 언어 설정
SystemMessage("항상 한국어로만 답변하세요. 영어 질문도 한국어로 답하세요.")
```